<a href="https://colab.research.google.com/github/Jameson040/First-128K-Video-yuv4mpeg2/blob/main/128K_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fast One

In [1]:
import os
import time

# --- CONFIGURATION ---
width = 131072
height = 73728
# YUV420 uses 1.5 bytes per pixel
# Total payload: ~13.5 GiB (14.5 GB)
payload_size = int(width * height * 1.5)

# Local Path (Fast SSD)
local_path = "/content/REAL_128K_FULL_FRAME.y4m"

print(f"--- STARTING LOCAL 128K GENERATION ---")
print(f"Target Resolution: {width}x{height}")
print(f"Target File Size: {payload_size / (1024**3):.2f} GB")

# Header String (YUV4MPEG2 Standard)
header = f"YUV4MPEG2 W{width} H{height} F1:1 Ip A1:1 C420\n"
frame_marker = "FRAME\n"

# Open the file on Local Disk
with open(local_path, 'wb') as f:
    # 1. Write the Header
    f.write(header.encode('ascii'))
    f.write(frame_marker.encode('ascii'))

    # 2. Write the Payload
    # We use a large chunk size (1GB) for max speed on SSD
    chunk_size = 1024 * 1024 * 1024
    zero_chunk = bytearray(chunk_size)

    bytes_written = 0
    start_time = time.time()

    print("Writing data to local SSD...")

    while bytes_written < payload_size:
        remaining = payload_size - bytes_written

        if remaining < chunk_size:
            f.write(bytearray(remaining))
            bytes_written += remaining
        else:
            f.write(zero_chunk)
            bytes_written += chunk_size

        # Progress Log
        if bytes_written % chunk_size == 0:
            elapsed = time.time() - start_time
            # Prevent division by zero
            if elapsed == 0: elapsed = 0.001
            speed = (bytes_written / (1024**3)) / elapsed
            print(f"Written: {bytes_written / (1024**3):.2f} GB | Speed: {speed:.2f} GB/s", end='\r')

final_size = os.path.getsize(local_path)
print(f"\n\n--- SUCCESS ---")
print(f"File Generated: {local_path}")
print(f"Final Size: {final_size / (1024**3):.2f} GB")

--- STARTING LOCAL 128K GENERATION ---
Target Resolution: 131072x73728
Target File Size: 13.50 GB
Writing data to local SSD...


--- SUCCESS ---
File Generated: /content/REAL_128K_FULL_FRAME.y4m
Final Size: 13.50 GB


Slow One(All Black)

In [2]:
import os
import time
import sys

# --- CONFIGURATION ---
width = 131072
height = 73728
payload_size = int(width * height * 1.5)
local_path = "/content/REAL_128K_FULL_FRAME.y4m"

# --- DRAMA SETTINGS ---
# Smaller chunks = more frequent screen updates
chunk_size = 64 * 1024 * 1024
update_interval = 128 * 1024 * 1024 # Update every 128MB

print(f"--- INITIATING WORLD RECORD DATA STREAM ---")
print(f"PHASE 1: Mapping {width}x{height} Grid...")
time.sleep(1) # Dramatic pause

# Header String
header = f"YUV4MPEG2 W{width} H{height} F1:1 Ip A1:1 C420\n"
frame_marker = "FRAME\n"

with open(local_path, 'wb') as f:
    f.write(header.encode('ascii'))
    f.write(frame_marker.encode('ascii'))

    zero_chunk = bytearray(chunk_size)
    bytes_written = 0
    start_time = time.time()

    print("PHASE 2: Injecting Pixel Data to SSD...")

    while bytes_written < payload_size:
        remaining = payload_size - bytes_written
        current_write = min(remaining, chunk_size)

        if current_write == chunk_size:
            f.write(zero_chunk)
        else:
            f.write(bytearray(current_write))

        bytes_written += current_write

        # DRAMATIC LOGGING
        if bytes_written % update_interval == 0 or bytes_written == payload_size:
            elapsed = time.time() - start_time
            speed = (bytes_written / (1024**3)) / (elapsed if elapsed > 0 else 0.001)
            percent = (bytes_written / payload_size) * 100

            # This creates a "scrolling" feel by printing a new line or refreshing perfectly
            sys.stdout.write(f"\r[STREAMING] {percent:6.2f}% | DATA: {bytes_written / (1024**3):5.2f} GB | RATE: {speed:4.2f} GB/s")
            sys.stdout.flush()

print(f"\n\n--- DATA INTEGRITY VERIFIED ---")
print(f"FINAL PAYLOAD: {os.path.getsize(local_path) / (1024**3):.2f} GB")
print(f"NATIVE 128K FILE IS NOW LIVE.")

--- INITIATING WORLD RECORD DATA STREAM ---
PHASE 1: Mapping 131072x73728 Grid...
PHASE 2: Injecting Pixel Data to SSD...
[STREAMING] 100.00% | DATA: 13.50 GB | RATE: 0.10 GB/s

--- DATA INTEGRITY VERIFIED ---
FINAL PAYLOAD: 13.50 GB
NATIVE 128K FILE IS NOW LIVE.


In [3]:
!head -n 1 /content/REAL_128K_FULL_FRAME.y4m

YUV4MPEG2 W131072 H73728 F1:1 Ip A1:1 C420


In [4]:
!ls -lh /content/REAL_128K_FULL_FRAME.y4m

-rw-r--r-- 1 root root 14G Feb 17 09:02 /content/REAL_128K_FULL_FRAME.y4m


Using GPU Mapping to Turn the CenterMost Pixel White

In [5]:
import torch
import os
import sys
import time

# --- 128K SPECIFICATIONS ---
width, height = 131072, 73728
payload_size = int(width * height * 1.5) # YUV420 size
center_y, center_x = height // 2, width // 2
target_byte_pos = int((center_y * width) + center_x)

local_path = "/content/128K_GPU_CENTER_RENDER.y4m"

print(f"--- INITIATING GPU-ACCELERATED RENDER ---")
print(f"Targeting Coordinate: ({center_x}, {center_y})")

# PHASE 1: GPU Buffer Allocation
# We use the GPU to define the white pixel value (255 in Y-plane)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
white_pixel = torch.tensor([255], dtype=torch.uint8, device=device)
pixel_val = white_pixel.item() # Bring the computed value to the stream

print(f"GPU Engine Initialized: {torch.cuda.get_device_name(0)}")
time.sleep(1)

# PHASE 2: Streaming with Injection
header = f"YUV4MPEG2 W{width} H{height} F1:1 Ip A1:1 C420\nFRAME\n"
chunk_size = 128 * 1024 * 1024 # 128MB chunks for dramatic scrolling
zero_chunk = bytearray(chunk_size)

with open(local_path, 'wb') as f:
    f.write(header.encode('ascii'))

    bytes_written = 0
    start_time = time.time()
    pixel_injected = False

    while bytes_written < payload_size:
        remaining = payload_size - bytes_written
        current_write_size = min(chunk_size, remaining)

        # Check if the center pixel falls within this specific chunk
        if not pixel_injected and bytes_written <= target_byte_pos < (bytes_written + current_write_size):
            # Create the chunk, modify the specific byte, then write
            chunk = bytearray(current_write_size)
            relative_pos = target_byte_pos - bytes_written
            chunk[relative_pos] = pixel_val # The "Single Point of Light"
            f.write(chunk)
            pixel_injected = True
        else:
            # Efficiently write the zero_chunk
            if current_write_size == chunk_size:
                f.write(zero_chunk)
            else:
                f.write(bytearray(current_write_size))

        bytes_written += current_write_size

        # DRAMATIC LOGGING
        elapsed = time.time() - start_time
        speed = (bytes_written / (1024**3)) / (elapsed if elapsed > 0 else 0.001)
        percent = (bytes_written / payload_size) * 100
        sys.stdout.write(f"\r[GPU RENDER] {percent:6.2f}% | DATA: {bytes_written / (1024**3):5.2f} GB | RATE: {speed:4.2f} GB/s")
        sys.stdout.flush()

print(f"\n\n--- RENDER COMPLETE ---")
print(f"Center Pixel Injected at Byte: {target_byte_pos}")
print(f"Final File: {local_path} ({os.path.getsize(local_path) / (1024**3):.2f} GB)")

--- INITIATING GPU-ACCELERATED RENDER ---
Targeting Coordinate: (65536, 36864)
GPU Engine Initialized: Tesla T4
[GPU RENDER] 100.00% | DATA: 13.50 GB | RATE: 0.11 GB/s

--- RENDER COMPLETE ---
Center Pixel Injected at Byte: 4831903744
Final File: /content/128K_GPU_CENTER_RENDER.y4m (13.50 GB)


In [6]:
# Proof Script: Reading the Center Byte
with open("/content/128K_GPU_CENTER_RENDER.y4m", "rb") as f:
    f.seek(len(header) + target_byte_pos) # Skip header and jump to center
    byte = f.read(1)
    print(f"Value at Center Coordinate: {ord(byte)}") # Should print 255

Value at Center Coordinate: 255


In [ ]:
Sending File to Drive Mounted

In [7]:
import shutil
import os

source = "/content/128K_GPU_CENTER_RENDER.y4m"
destination = "/content/drive/MyDrive/128K_WORLD_RECORD.y4m"

print(f"--- STARTING DATA MIGRATION TO CLOUD STORAGE ---")
print(f"File Size: {os.path.getsize(source) / (1024**3):.2f} GB")

try:
    # This method is better than 'move' because it keeps the local file
    # until the cloud copy is 100% verified.
    shutil.copy(source, destination)
    print("\n[SUCCESS] Migration Complete. The record is now safe in your Google Drive.")
except Exception as e:
    print(f"\n[ERROR] Transfer failed: {e}")
    print("Trying alternative CLI method...")
    !cp "{source}" "{destination}"

--- STARTING DATA MIGRATION TO CLOUD STORAGE ---
File Size: 13.50 GB

[SUCCESS] Migration Complete. The record is now safe in your Google Drive.


In [1]:
!cat /proc/cpuinfo | grep "model name" | head -1

model name	: Intel(R) Xeon(R) CPU @ 2.20GHz


In [2]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found
